In [1]:
import sys
import pandas as pd



# Split the tree into two given the column and value to be used for splitting
def split_data(column, value, data):
    print("Calling split_data")
    columns=data.columns.values
    left, right = pd.DataFrame(columns=columns),pd.DataFrame(columns=columns)
    left=data.loc[data[column]<=value]
    right=data.loc[data[column]>value]
    del data
    return left, right


# Calculate the Gini index for a split dataset
# def gini_index(groups, classes):
#     # count all samples at split point
#     n_instances = float(sum([len(group) for group in groups]))
#     # sum weighted Gini index for each group
#     gini = 0.0
#     for group in groups:
#         size = float(len(group))
#         # avoid divide by zero
#         if size == 0:
#             continue
#         score = 0.0
#         # score the group based on the score for each class
#         for class_val in classes:
#             p = [row[-1] for row in group].count(class_val) / size
#             score += p * p
#         # weight the group score by its relative size
#         gini += (1.0 - score) * (size / n_instances)
#     return gini


# Calculate the Cost function for a split dataset
def mean_square_error(lc, rc):
	print("Calling mean_sqr_error")
	mse=0;
	n_tot=len(lc)+len(rc)
	group_sum_sqr=0

	mean_lc=lc.mean(axis=0)[8];
	for row in lc.iterrows():
		group_sum_sqr+=(row[1]["output"]-mean_lc)**2;
	mse+=len(lc)*group_sum_sqr;

	group_sum_sqr=0
	mean_rc=rc.mean(axis=0)[8];
	for row in rc.iterrows():
		group_sum_sqr+=(row[1]["output"]-mean_rc)**2;
	mse+=len(rc)*group_sum_sqr;

	mse/=n_tot
	return mse

def absolute_error(lc, rc):
	print("Calling absolute_error")
	ae=0;
	
	mean=lc.mean(axis=0)[8];
	for row in lc.iterrows():
		ae+=abs(row[1]["output"]-mean)

	mean=rc.mean(axis=0)[8];
	for row in rc.iterrows():
		ae+=abs(row[1]["output"]-mean)
	return ae


# Select the best split value for the dataset returns the node ( containing the split value column, and the two groups)
def best_split(data, loss_function):
    print("Calling best_split")
    best_column, best_value, best_cost, best_groups = None, float('inf'), float("inf"), None
    for column in data.columns.values[:-1]:
        for row in data.iterrows():
            left_child, right_child = split_data(column, row[1][column], data)  # calls split data properly
            if(loss_function=="absolute"):
                cost = absolute_error(left_child, right_child)
            elif(loss_function=="mean_squared"):
                cost = mean_square_error(left_child, right_child)

        if cost < best_cost:
            best_column, best_value, best_cost, best_lc, best_rc = column, row[1][column], cost, left_child, right_child
    print(best_column,best_value, best_lc.head(), best_rc.head())
    return {'column': best_column, 'value': best_value, 'left_child': best_lc, 'right_child': best_rc}



# Create a terminal node value
def create_leaf(group):
	print("Calling create_leaf")
	#nodeval = sum([g[-1] for g in group]) / len(group)
    #return nodeval
	mean=group.mean(axis=0)[8];
	return {'column': None, 'value': None, 'left_child': None, 'right_child': None, 'prediction':mean}


# Spliting the tree  recursively at a particular node or fixing a terminal
def recursive_split(node, max_depth, min_records, cur_depth, loss_func):
    print("Calling recursive_split")
    left, right = node['left_child'], node['right_child']


    # if the best split is a no split
    if len(left)==0 or len(right)==0:
        print("No split")
        node['left_child'] = node['right_child'] = create_leaf(left.append(right))
        return

    # check for max depth
    if cur_depth >= max_depth:
        print("Max depth reached")
        node['left_child'], node['right_child'] = create_leaf(left), create_leaf(right)
        return

    # split left side if greater than minimum records
    if len(left) <= min_records:
        print("left_child_min_nodes_reached")
        node['left_child'] = create_leaf(left)
    else:
        node['left_child'] = best_split(left, loss_func)
        recursive_split(node['left_child'], max_depth, min_records, cur_depth + 1, loss_func)

    # split right side if greater than minimum records
    if len(right) <= min_records:
        print("right_child_min_nodes_reached")
        node['right_child'] = create_leaf(right)
    else:
        node['right_child'] = best_split(right, loss_func)
        recursive_split(node['right_child'], max_depth, min_records, cur_depth + 1, loss_func)


# Build a decision tree
def build_tree(dataset, max_depth, min_records, loss_func):
	print("Calling build_tree")
	root_node = best_split(dataset, loss_func)
	recursive_split(root_node, max_depth, min_records, 1, loss_func)
	return root_node


# Predict the output of a data value using the decision tree
def predict_output(node, test_record):
	print("Calling predict_output")
	if test_record[node['column']] < node['value']:  # checks if belongs to node's left or right 
		if isinstance(node['left'], dict):  # check if its dictionary or not, dictionary means it has subnodes
			predict_output(node['left'], test_record)
		else:
			return node['left']  # its a terminal node return it
	else:
		if isinstance(node['right'], dict):
			predict_output(node['right'], test_record)
		else:
			return node['right']


# Print the tree structure
def print_tree(node, depth=0):
	if node['left_child']!=None and node['right_child']!=None:
		print('%s[X%s < %.3f]' % (depth * ' ', (node['column']), node['value']))
		print_tree(node['left_child'], depth + 1)
		print_tree(node['right_child'], depth + 1)
	else:
		print('%s[%s]' % ((depth * ' ', node)))

In [2]:
train_data=pd.read_csv("train.csv", header=0)

In [3]:
train_data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,output
0,0.63,498.55,308.70,271.7,4.5,2,0.40,5,17.15
1,0.72,432.08,360.15,181.1,8.0,2,0.10,2,42.32
2,0.58,531.79,360.15,271.7,4.5,2,0.40,4,22.35
3,0.72,432.08,360.15,181.1,8.0,2,0.25,2,45.20
4,0.75,415.46,334.43,181.1,8.0,3,0.10,5,25.53


In [5]:
rootnode = build_tree(train_data[0:50], 6, 1, "mean_squared")

Calling build_tree
Calling best_split
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_er

Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sq

Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
X5 4.5      X1      X2      X3     X4   X5  X6    X7  X8  output
0  0.63  498.55  308.70  271.7  4.5   2  0.40   5   17.15
2  0.58  531.79  360.15  271.7  4.5   2  0.40   4   22.35
5  0.65  481.93  282.98  271.7  4.5   2  0.25   2   15.38
6  0.67  465.31  257.25  271.7  4.5   5  0.25   1   15.99
9  0.57  548.41  385.88  271.7  4.5   2  0.40   2   17.90      X1      X2      X3     X4   X5  X6    X7  X8  output
1  0.72  432.08  360.15  181.1  8.0   2  0.10   2   42.32
3  0.72  432.08  360.15  181.1  8.0   2  0.25   2   45.20
4  0.75  415.46  334.43  181.1  8.0   3  0.10   5   25.53
7  0.82  382.22  334.43  150.9  8.0   3  0.40   1   37.49
8  0.69  448.70  437.33  150.9  8.0   3  0.25   2   38.49
Calling recursive_split
Calling best_split
Ca

Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sq

Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
X1 0.65       X1      X2      X3     X4   X5  X6    X7  X8  output
48  0.65  481.93  282.98  271.7  4.5   3  0.25   1   16.35      X1      X2      X3     X4   X5  X6    X7  X8  output
6  0.67  465.31  257.25  271.7  4.5   5  0.25   1   15.99
Calling recursive_split
left_child_min_nodes_reached
Calling create_leaf
right_child_min_nodes_reached
Calling create_leaf
Calling best_split
Calli

...

Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
X7 0.1       X1      X2      X3     X4   X5  X6   X7  X8  output
1   0.72  432.08  360.15  181.1  8.0   2  0.1   2   42.32
4   0.75  415.46  334.43  181.1  8.0   3  0.1   5   25.53
11  0.75  415.46  334.43  181.1  8.0   3  0.1   3   28.39
27  0.72  432.08  360.15  181.1  8.0   3  0.1   4   44.62
30  0.72  432.08  360.15  181.1  8.0   5  0.1   3   35.34       X1      X2      X3     X4   X5  X6    X7  X8  output
3   0.72  432.08  360.15  181.1  8.0   2  0.25   2   45.20
7   0.82  382.22  334.43  150.9  8.0   3  0.40   1   37.49
8   0.69  448.70  437.33  150.9  8.0   3  0.25   2   38.49
10  0.89  348.99  308.70  135.8  8.0   2  0.40   2   33.97
12  0.72  432.08  360.15  181.1  8.0   3  0.40   5   38.85
Calling recursive_split
Calling best_split
Calling split_d

33  0.82  382.22  334.43  150.9  8.0   2  0.1   2   33.54
Calling recursive_split
left_child_min_nodes_reached
Calling create_leaf
right_child_min_nodes_reached
Calling create_leaf
Calling best_split
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Ca

Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sq

X8 2.0       X1      X2      X3     X4   X5  X6   X7  X8  output
16  0.75  415.46  334.43  181.1  8.0   2  0.4   1   32.68
43  0.75  415.46  334.43  181.1  8.0   5  0.4   2   33.93       X1      X2      X3     X4   X5  X6    X7  X8  output
31  0.75  415.46  334.43  181.1  8.0   2  0.25   5   27.35
35  0.75  415.46  334.43  181.1  8.0   3  0.25   5   26.88
Calling recursive_split
Calling best_split
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling spl

Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sqr_error
Calling split_data
Calling mean_sq

In [6]:
print_tree(rootnode)

[XX5 < 4.500]
 [XX2 < 481.930]
  [XX7 < 0.000]
   [{'column': None, 'value': None, 'left_child': None, 'right_child': None, 'prediction': 12.04}]
   [XX8 < 1.000]
    [XX1 < 0.650]
     [{'column': None, 'value': None, 'left_child': None, 'right_child': None, 'prediction': 16.35}]
     [{'column': None, 'value': None, 'left_child': None, 'right_child': None, 'prediction': 15.99}]
    [XX7 < 0.100]
     [XX1 < 0.650]
      [{'column': None, 'value': None, 'left_child': None, 'right_child': None, 'prediction': 14.57}]
      [{'column': None, 'value': None, 'left_child': None, 'right_child': None, 'prediction': 14.57}]
     [XX2 < 465.310]
      [{'column': None, 'value': None, 'left_child': None, 'right_child': None, 'prediction': 14.47}]
      [{'column': None, 'value': None, 'left_child': None, 'right_child': None, 'prediction': 15.275}]
  [XX6 < 2.000]
   [XX2 < 531.790]
    [XX1 < 0.580]
     [XX7 < 0.250]
      [{'column': None, 'value': None, 'left_child': None, 'right_child': None

In [69]:
# Predict the output of a data value using the decision tree
def predict_output(node, test_record):
    
    if node['value']==None:
        
        return node['prediction']
    
    if test_record[node['column']] < node['value']:  # checks if belongs to node's left or right child
        
        return predict_output(node['left_child'], test_record)
       
    else:
        
        return predict_output(node['right_child'], test_record)
        
            


In [70]:
record=train_data.loc[0]
print(predict_output(rootnode,record))

38.49


In [87]:
output=pd.DataFrame(columns=['Id','output'])
i=int(1)
for row in test_data.iterrows():
    output=output.append({'Id':int(i), 'output':predict_output(rootnode, row[1])}, ignore_index=True)
    if i%5==0:
        print(i)
    i+=int(1)
output['Id']=output['Id'].astype(int)

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190


In [80]:
test_data=pd.read_csv("test.csv", header=0)

In [85]:
output['Id'].astype(int)

0        1
1        2
2        3
3        4
4        5
5        6
6        7
7        8
8        9
9       10
10      11
11      12
12      13
13      14
14      15
15      16
16      17
17      18
18      19
19      20
20      21
21      22
22      23
23      24
24      25
25      26
26      27
27      28
28      29
29      30
      ... 
162    163
163    164
164    165
165    166
166    167
167    168
168    169
169    170
170    171
171    172
172    173
173    174
174    175
175    176
176    177
177    178
178    179
179    180
180    181
181    182
182    183
183    184
184    185
185    186
186    187
187    188
188    189
189    190
190    191
191    192
Name: Id, Length: 192, dtype: int32

In [88]:
output

,Id,output
0,1,38.850
1,2,38.490
2,3,30.455
3,4,38.490
4,5,38.490
5,6,38.490
6,7,38.490
7,8,38.490
8,9,34.360
9,10,35.380
